In [ ]:
prompt = """[Background] Human values are the principles, standards, and qualities considered worthwhile or desirable by people. Schwartz values include self-direction, stimulation, hedonism, achievement, power, security, tradition, conformity, benevolence, and universalism.
[Task] Write a story of over 1000 words and involving multiple characters, each of whom represents unique Schwartz values. The story can manifest the values that the characters support and oppose. After writing the story, conclude the values each character supports and opposes.
[Output format] The output follows the below JSON format:
{
  "story": "The story text",
  "characters": [
    {
      "name": "Alice",
      "supports": ["self-direction", "hedonism"],
      "opposes": ["conformity"]
    },
    {
      "name": "Bob",
      "supports": ["achievement", "power"],
      "opposes": ["tradition"]
    }
  ]
}
"""

from gpv.models import LLMModel

model_name = "gpt-4o"
model = LLMModel(model_name, temperature=1.0, max_tokens=4096)

responses = model([prompt], n=2)
